In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import KFold,StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, PowerTransformer, RobustScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

from keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import (mean_squared_error,
                             r2_score,
                             mean_absolute_percentage_error,
                             mean_absolute_error
                            )

from sklearn.pipeline import Pipeline
from statsmodels.tsa.api import VAR

from sklearn.feature_selection import SelectKBest,f_regression

In [2]:
df = pd.read_csv('../../data/dataset/Full_data_File.csv')

In [3]:
df.head()

,location,country,latitude,longitude,generationtime_ms,elevation,Date,year,month,day,...,RH,Dew_Point,snowfall,Pressure,wind_speed,wind_direction,wind_gusts,sunshine_duration,direct_radiation,AQI
0,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-01,2021,1,1,...,92.208333,1.266667,0.0,1006.029167,7.208333,243.250000,11.187500,890.836250,16.208333,65.125000
1,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-02,2021,1,2,...,96.708333,1.312500,0.0,1011.245833,6.445833,216.916667,10.283333,937.792917,22.625000,33.291667
2,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-03,2021,1,3,...,91.458333,1.229167,0.0,1015.504167,14.333333,52.208333,25.516667,739.805000,17.791667,34.041667
3,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-04,2021,1,4,...,86.291667,0.879167,0.0,1017.016667,25.041667,49.375000,45.595833,0.000000,1.041667,19.750000
4,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-05,2021,1,5,...,91.125000,1.475000,0.0,1016.929167,18.708333,54.541667,34.145833,46.027083,1.833333,10.166667


In [4]:
df.shape

(49674, 27)

In [5]:
def get_important_Feature(df):
    
    def get_train_test(df):
        X = df.iloc[:,0:-1]
        y = df.iloc[:,-1]

        X = MinMaxScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        return X_train, X_test, y_train, y_test
    
    def fitness_function(selected_features, X_train, X_test, y_train, y_test):
        # Select features based on the particle's position
        selected_features = np.array(selected_features, dtype=bool)
        if not any(selected_features):  # Avoid empty feature selection
            return float('inf')  # Assign a very high MSE

        X_train_selected = X_train[:, selected_features]
        X_test_selected = X_test[:, selected_features]

        # Train a regression model
        model = DecisionTreeRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        predictions = model.predict(X_test_selected)

        # Calculate and return MSE
        mse = mean_squared_error(y_test, predictions)
        return mse
    
    def pso_feature_selection(n_particles, n_iterations, n_features, df):
        # Initialize particles
        particles = [np.random.choice([0, 1], size=n_features) for _ in range(n_particles)]
        velocities = [np.random.uniform(-1, 1, size=n_features) for _ in range(n_particles)]
        personal_best_positions = particles.copy()
        personal_best_scores = [float('inf')] * n_particles
        global_best_position = None
        global_best_score = float('inf')

        X_train, X_test, y_train, y_test = get_train_test(df)

        # PSO loop
        for iteration in range(n_iterations):
            for i, particle in enumerate(particles):
                # Evaluate fitness
                mse = fitness_function(particle, X_train, X_test, y_train, y_test)

                # Update personal best
                if mse < personal_best_scores[i]:
                    personal_best_scores[i] = mse
                    personal_best_positions[i] = particle.copy()

                # Update global best
                if mse < global_best_score:
                    global_best_score = mse
                    global_best_position = particle.copy()

            # Update particle velocities and positions
            for i, particle in enumerate(particles):
                r1, r2 = np.random.rand(n_features), np.random.rand(n_features)
                cognitive = r1 * (personal_best_positions[i] - particle)
                social = r2 * (global_best_position - particle)
                velocities[i] = 0.5 * velocities[i] + 1.5 * cognitive + 1.5 * social
                particles[i] = np.where(np.random.rand(n_features) < 1 / (1 + np.exp(-velocities[i])), 1, 0)  # Sigmoid activation

            # Print progress
#             print(f"Iteration {iteration+1}/{n_iterations}, Best MSE: {global_best_score}")

        return global_best_position, global_best_score



    n_particles = 20
    n_iterations = 50
    n_features = len(list(df)) - 1

    # Run PSO
    best_features, best_score = pso_feature_selection(n_particles, n_iterations, n_features, df)

    best_feature_list = []
    for name_, flag in zip(list(df)[:-1],best_features):
        if flag == 1:
            best_feature_list.append(name_)
    best_feature_list.append(list(df)[-1])
    return df[best_feature_list], best_feature_list

In [6]:
def get_k_fold_operation_data_proposed_approach(df, date_, cv):
    
    df = pd.DataFrame(df)
    colLen = len(list(df))

    X = df.values[:, 0:colLen - 1]
    y = df.values[:, colLen - 1]
    
    
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    date_train_list = []
    date_test_list = []
    
    kf = KFold(n_splits=cv)
    
    for train_index,test_index in kf.split(X,y):
        X_train, X_test, y_train, y_test, date_train, date_test  = X[train_index], X[test_index], y[train_index], y[test_index], date_[train_index], date_[test_index]
        
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)
        
        date_train_list.append(date_train)
        date_test_list.append(date_test)
        
    
        
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list

In [7]:
def get_IQR_Operation_value(df, col, cv):
    
    S_B = df.shape[0]
    
    Q1 = df['AQI'].quantile(0.25)
    Q3 = df['AQI'].quantile(0.75)

    # Calculate IQR
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.0 * IQR
    upper_bound = Q3 + 1.75 * IQR
    
    print(lower_bound,upper_bound)
    
#     df = df.loc[(df['AQI'] <= upper_bound) & (df['AQI'] >= lower_bound)]
    date_ = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day)).values
    df, impt_col = get_important_Feature(df)

    X_train_list, \
    X_test_list, \
    y_train_list, \
    y_test_list, \
    date_train_list, \
    date_test_list = get_k_fold_operation_data_proposed_approach(df, date_, cv)
    
    S_A = df.shape[0]
    
    diff = S_B - S_A
    
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list, diff, impt_col
    

In [8]:
def get_k_fold_operation_data(df, cv):
    
    df = pd.DataFrame(df)
    colLen = len(list(df))

    X = df.values[:, 0:colLen - 1]
    y = df.values[:, colLen - 1]
    date_ = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day)).values
    
    
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    date_train_list = []
    date_test_list = []
    
    kf = KFold(n_splits=cv)
    
    for train_index,test_index in kf.split(X,y):
        X_train, X_test, y_train, y_test, date_train, date_test  = X[train_index], X[test_index], y[train_index], y[test_index], date_[train_index], date_[test_index]
        
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)
        
        date_train_list.append(date_train)
        date_test_list.append(date_test)
        
    
        
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, PowerTransformer, RobustScaler
import numpy as np
import tensorflow as tf

def GRU_Regression(X_train, X_test, y_train, y_test, scaler_name):
    # Initialize the scaler
    scaler = None
    if scaler_name == 'StandardScaler':
        scaler = StandardScaler()
    elif scaler_name == 'Normalizer':
        scaler = Normalizer()
    elif scaler_name == 'MinMaxScaler':
        scaler = MinMaxScaler()
    elif scaler_name == 'PowerTransformer':
        scaler = PowerTransformer(method='yeo-johnson')
    elif scaler_name == 'RobustScaler':
        scaler = RobustScaler()

    # Apply scaling
    if scaler_name == 'not_processing':
        X_train_scaled = X_train
        X_test_scaled = X_test
    else:
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    
    # Reshape input data for GRU (samples, timesteps, features)
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    # Set random seeds for reproducibility
    np.random.seed(1)
    tf.keras.utils.set_random_seed(1)

    # Build the GRU model
    def build_gru(input_shape):
        model = Sequential([
            Input(shape=input_shape),
            GRU(64, activation='relu', return_sequences=True),
            GRU(32, activation='relu'),
            Dense(1, activation='linear')  # Regression output layer
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    gru_model = build_gru((X_train_scaled.shape[1], X_train_scaled.shape[2]))

    # Train the GRU model
    gru_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    # Predict using the GRU model
    y_pred = gru_model.predict(X_test_scaled).flatten()

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100

    return mae, rmse, r_2, mape, y_pred


In [10]:
summary_value = []
detail_value = []

for city_ in df['location'].unique():
    city_df = df[df['location'] == city_]
    country_ = city_df['country'].unique()[0]
    
    print(city_)
    city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)
    
    target_col = list(city_df)[-1]
#     city_df = get_IQR_Operation_value(city_df,target_col)
    
    X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list = get_k_fold_operation_data(city_df, 5)
    
    k_fold = 1
    
    mae_list = []
    rmse_list = []
    r_2_list = []
    mape_list = []
    
    for X_train, X_test, y_train, y_test, date_train, date_test in zip(X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list):
        for _preprocessing_name in ['StandardScaler']: #'RobustScaler', 'MinMaxScaler' , 'StandardScaler'
            
            res = []
            
            mae, rmse, r_2, mape, y_pred = GRU_Regression(X_train, X_test, y_train, y_test, _preprocessing_name)

            res.append(country_)
            res.append(city_)
            res.append(mae)
            res.append(rmse)
            res.append(r_2)
            res.append(mape)
            res.append('fold_' + str(k_fold))
            res.append(_preprocessing_name)
            res.append('GRU')
            print(res)

            full_output = pd.DataFrame({
                'actual_value' : y_test,
                'predict_value' : y_pred,
                'Fold_number' : ['fold_' + str(k_fold)] * len(y_test),
                'date' : date_test
            })
            full_output['city'] = city_
            full_output['country'] = country_
            full_output['pre-processing name'] = _preprocessing_name
            full_output['model_name'] = 'GRU'

            summary_value.append(res)
            detail_value.append(full_output)

        k_fold = k_fold + 1
        
        

Amsterdam


C:\Users\DELL\AppData\Local\Temp\ipykernel_8948\2602270106.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 [==============================] - 0s 1ms/step
['Netherlands', 'Amsterdam', 1.1445713715884196, 1.4937299773992698, 0.9675191625082006, 4.648965512237127, 'fold_1', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Netherlands', 'Amsterdam', 1.1447487835470398, 1.5126915420043159, 0.9770633143996257, 4.754560989139997, 'fold_2', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 958us/step
['Netherlands', 'Amsterdam', 1.1988359923776424, 1.574353440638076, 0.9687339075280654, 4.937538039401339, 'fold_3', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Netherlands', 'Amsterdam', 1.3286340127796887, 1.724435436960069, 0.9432423948741446, 5.876213912789383, 'fold_4', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Netherlands', 'Amsterdam', 1.455102010404683, 1.840543826581489, 0.9566662370780725, 5.726627392185194, 'fold_5', 'StandardScaler', 'GRU']
Athens


C:\Users\DELL\AppData\Local\Temp\ipykernel_8948\2602270106.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 [==============================] - 0s 1ms/step
['Greece', 'Athens', 1.4792050219504362, 1.8959547036556879, 0.9659878722798715, 4.047543521341114, 'fold_1', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Greece', 'Athens', 1.50848530852087, 1.8759878362172837, 0.9549470378911326, 4.470924336565601, 'fold_2', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 2ms/step
['Greece', 'Athens', 1.7424459065476503, 3.1365113695420015, 0.8987251695983054, 5.3705861863944815, 'fold_3', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Greece', 'Athens', 1.901026534163244, 2.4832225326770847, 0.913011533779423, 5.769381076817136, 'fold_4', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Greece', 'Athens', 1.7328248524774696, 2.3028008183499304, 0.9457381721107031, 5.060414703587226, 'fold_5', 'StandardScaler', 'GRU']
Barcelona


...

10/10 [==============================] - 0s 1ms/step
['Sewden', 'Stockholm', 1.124115075945719, 1.477036932617825, 0.9329558128141588, 5.288980974268649, 'fold_1', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Sewden', 'Stockholm', 0.8492394375474484, 1.2760234753416786, 0.9556610129932819, 3.7795924177272586, 'fold_2', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Sewden', 'Stockholm', 1.1935928553751067, 1.684137759424517, 0.92633638147605, 5.554950729369811, 'fold_3', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Sewden', 'Stockholm', 1.0371452542744817, 1.2753064443654822, 0.9197139211093683, 5.154935413587568, 'fold_4', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Sewden', 'Stockholm', 0.9700613130717518, 1.197866797707554, 0.9332317472237188, 4.080399139553603, 'fold_5', 'StandardScaler', 'GRU']
bern


C:\Users\DELL\AppData\Local\Temp\ipykernel_8948\2602270106.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 [==============================] - 0s 1ms/step
['Switzerland', 'bern', 1.0987430568169954, 1.8498923424997684, 0.9318246353857587, 4.047720273855752, 'fold_1', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Switzerland', 'bern', 1.1659623781840007, 1.555427773108411, 0.9675012790897659, 4.579396439763818, 'fold_2', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Switzerland', 'bern', 0.9501890580948081, 1.2474606483740829, 0.976889227567799, 4.013404710744119, 'fold_3', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Switzerland', 'bern', 1.0907818704979604, 1.4086489046294945, 0.9658372770452137, 4.847050717905272, 'fold_4', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Switzerland', 'bern', 1.815785666034647, 2.1072575005641423, 0.8934148200642487, 7.585046416754233, 'fold_5', 'StandardScaler', 'GRU']
luxembourg findel airport


C:\Users\DELL\AppData\Local\Temp\ipykernel_8948\2602270106.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 [==============================] - 0s 1ms/step
['Luxembourg', 'luxembourg findel airport', 1.2068239327041232, 1.862359743931155, 0.9365192215778263, 4.957861945533325, 'fold_1', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Luxembourg', 'luxembourg findel airport', 1.0277456181234423, 1.3470223359382718, 0.9709464218374316, 4.698858722327948, 'fold_2', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Luxembourg', 'luxembourg findel airport', 0.975311092045753, 1.2551585634436593, 0.9762117674537538, 4.36082863738899, 'fold_3', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Luxembourg', 'luxembourg findel airport', 1.0551155375563392, 1.3261472375568106, 0.9666713213365955, 4.735165446403314, 'fold_4', 'StandardScaler', 'GRU']
10/10 [==============================] - 0s 1ms/step
['Luxembourg', 'luxembourg findel airport', 1.5145847263945846, 1.783375359918419, 0.9437416090665274, 6.2

In [11]:
summary_df = pd.DataFrame(summary_value,columns=['Country','City','MAE','RMSE','R2','MAPE','K_Fold','pre-processing name','model_name'])

In [12]:
detail_df = pd.concat(detail_value)

In [13]:
summary_df.to_csv('../_output/AQI_GRU_Summary_result.csv',index=False)

In [14]:
detail_df.to_csv('../_output/AQI_GRU_Detail_result.csv',index=False)